Symbolic execution
1. Parse the code into a DAG
2. Collect all the conditions in every path
3. Solve it using Z3

In [14]:
import z3 as z3
import z3.z3


def getMaxValue(a,b,c):
    z_ = True
    z1 = z_ and False
    a = 1 - 3
    b = 3 - a
    c = 4
    a = a+1
    if b*b>c*c:
        print("Hello","World")
    else:
        print("bla")
    z1 = 2

In [15]:
import dis
dis.dis(getMaxValue)

  6           0 LOAD_CONST               1 (True)
              2 STORE_FAST               3 (z_)

  7           4 LOAD_FAST                3 (z_)
              6 JUMP_IF_FALSE_OR_POP     5 (to 10)
              8 LOAD_CONST               2 (False)
        >>   10 STORE_FAST               4 (z1)

  8          12 LOAD_CONST               3 (-2)
             14 STORE_FAST               0 (a)

  9          16 LOAD_CONST               4 (3)
             18 LOAD_FAST                0 (a)
             20 BINARY_SUBTRACT
             22 STORE_FAST               1 (b)

 10          24 LOAD_CONST               5 (4)
             26 STORE_FAST               2 (c)

 11          28 LOAD_FAST                0 (a)
             30 LOAD_CONST               6 (1)
             32 BINARY_ADD
             34 STORE_FAST               0 (a)

 12          36 LOAD_FAST                1 (b)
             38 LOAD_FAST                1 (b)
             40 BINARY_MULTIPLY
             42 LOAD_FAST                2

In [3]:
z = dis.Bytecode(getMaxValue)

In [31]:
from z3 import z3
for instr in z:
    xx = instr
    break

x1 = z3.Int('x1@0')
x2 = z3.Int('x2@0')
x3 = x2 + x1
x4 = x3 + 1
x5 = z3.Real('x5@0')

s = z3.Solver()
s1 = z3.Solver()
s.translate(z3.main_ctx())

# Mapping between variable name and the corresponding z3 instance
# Mapping between variable name and latest version
# Accumulate all the s
from collections import deque
d = deque()
d.append(1)
d.append(2)


def createBinaryOperationResult(variableType_: z3.ExprRef,
                                isInt: bool,
                                variableName: str,
                                mapVariableToItsLatestVersion: map
                                ):
    if variableName in mapVariableToItsLatestVersion:
        #Create a new version
        mapVariableToItsLatestVersion[variableName] += 1
    else:
        #First version
        mapVariableToItsLatestVersion[variableName] = 1
    match variableType_:
        case z3.ArithRef:
            if isInt:
                res = z3.Int(variableName+"@"+str(mapVariableToItsLatestVersion[variableName]))
            else:
                res = z3.Real(variableName+"@"+str(mapVariableToItsLatestVersion[variableName]))
        case z3.BoolRef:
            res = z3.Bool(variableName+"@"+str(mapVariableToItsLatestVersion[variableName]))
    return res


def handleBinaryInstruction(operationName: str,
                            variableName: str,
                            operandsStack: deque,
                            mapVariableToLatestVersionId: map,
                            mapVariableNameToLatestZ3Ref: map,
                            solver_: z3.Solver):
    operand1 = operandsStack.pop()
    operand2 = operandsStack.pop()

    #z3 supports Integers, Reals and Bools
    match operationName:
        case "BINARY_ADD":
            variableType = type(operand1+operand2)
            isInt = False
            if variableType == z3.ArithRef:
                temp = operand1 + operand2
                isInt = temp.is_int()
            res = createBinaryOperationResult(variableType,isInt,variableName,mapVariableToLatestVersionId)
            solver_.add(res == operand1 + operand2)
        case "BINARY_SUBTRACT":
            variableType = type(operand1-operand2)
            isInt = False
            if variableType == z3.ArithRef:
                temp = operand1 - operand2
                isInt = temp.is_int()
            res = createBinaryOperationResult(variableType,isInt,variableName,mapVariableToLatestVersionId)
            solver_.add(res == operand1 - operand2)
        case "BINARY_MULTIPLY":
            variableType = type(operand1*operand2)
            isInt = False
            if variableType == z3.ArithRef:
                isInt = (z3.ArithRef(operand1 * operand2)).is_int()
            res = createBinaryOperationResult(variableType,isInt,variableName,mapVariableToLatestVersionId)
            solver_.add(res == operand1 * operand2)

    mapVariableNameToLatestZ3Ref[variableName] = res
    operandsStack.append(res)
    return

# operation = instr_.opname
# variableName = str(instr.argval)
operand1 = z3.Int('a@1')
operand2 = z3.Int('b@1')

operandsStack = deque()
operandsStack.append(operand1)
operandsStack.append(operand2)

operationName = "BINARY_ADD"
mapVariableToLatestVersionId = {"a":1,"b":1}
mapVariableToLatestZ3Ref = {"a":operand1,"b":operand2}

variableName = "c"
s = z3.Solver()
# handleBinaryInstruction(operationName,variableName,operandsStack,mapVariableToLatestVersionId,mapVariableToLatestZ3Ref,s)


operand3 = operand1 + 1
operand3

a@1 + 1

In [10]:
edges = []
nodes = []
prevInstructionIsJump = False
for instr in z:
    # Adding edges
    if prevInstructionIsJump:
        edges.append((nodes[-1],instr.offset))
    if "JUMP" in instr.opname:
        edges.append((nodes[-1],int(instr.argval)))
    # Adding nodes
    if instr.is_jump_target:
        nodes.append(instr.offset)
    if prevInstructionIsJump:
        nodes.append(instr.offset)
    if instr.offset == 0:
        nodes.append(instr.offset)
    print(instr.offset,instr.opname,instr.arg,instr.argval,instr.is_jump_target)
    if "JUMP" in instr.opname:
        prevInstructionIsJump = True
    else:
        prevInstructionIsJump = False

nodes = list(set(nodes))
nodes.sort()
edges = list(set(edges))

print(nodes,edges)

0 LOAD_CONST 1 True False
2 STORE_FAST 3 z_ False
4 LOAD_FAST 3 z_ False
6 JUMP_IF_FALSE_OR_POP 5 10 False
8 LOAD_CONST 2 False False
10 STORE_FAST 4 z1 True
12 LOAD_CONST 3 1 False
14 STORE_FAST 0 a False
16 LOAD_CONST 4 3 False
18 STORE_FAST 1 b False
20 LOAD_CONST 5 4 False
22 STORE_FAST 2 c False
24 LOAD_FAST 0 a False
26 LOAD_CONST 3 1 False
28 BINARY_ADD None None False
30 STORE_FAST 0 a False
32 LOAD_FAST 1 b False
34 LOAD_FAST 1 b False
36 BINARY_MULTIPLY None None False
38 LOAD_FAST 2 c False
40 LOAD_FAST 2 c False
42 BINARY_MULTIPLY None None False
44 COMPARE_OP 4 > False
46 POP_JUMP_IF_FALSE 33 66 False
48 LOAD_GLOBAL 0 print False
50 LOAD_CONST 6 Hello False
52 LOAD_CONST 7 World False
54 CALL_FUNCTION 2 2 False
56 POP_TOP None None False
58 LOAD_CONST 9 2 False
60 STORE_FAST 4 z1 False
62 LOAD_CONST 0 None False
64 RETURN_VALUE None None False
66 LOAD_GLOBAL 0 print True
68 LOAD_CONST 8 bla False
70 CALL_FUNCTION 1 1 False
72 POP_TOP None None False
74 LOAD_CONST 9 2 False

In [ ]:
dis.dis(getMaxValue)

In [9]:
from dis import Instruction
from typing import List
import math
class Node:
    def __init__(self, nodeId_,instructions_: List[Instruction],children_: List['Node']=None):
        if children_ is None:
            children_ = []
        self.children = children_
        self.nodeId = nodeId_
        self.instructions = instructions_

nodeIdToGraphObjectMapping = {}
for i in range(len(nodes)):
    start = nodes[i]
    # If i is the last node
    if i+1==len(nodes):
        end = math.inf
    else:
        end = nodes[i+1]
    instructions = []
    nodeId = start
    for instr in z:
        if start <= instr.offset < end:
            instructions.append(instr)
    nodeIdToGraphObjectMapping[nodeId] = Node(nodeId,instructions)

for (nodeId1,nodeId2) in edges:
    nodeIdToGraphObjectMapping[nodeId1].children.append(nodeIdToGraphObjectMapping[nodeId2])

root = nodeIdToGraphObjectMapping[0]



NameError: name 'nodes' is not defined

In [ ]:
from graphviz import Digraph
gra = Digraph()
# print(nodeIdToGraphObjectMapping)
# str(nodeIdToGraphObjectMapping[0].instructions
for (nodeId,node) in nodeIdToGraphObjectMapping.items():
    gra.node(str(nodeId),str(",".join([str(instr.offset) for instr in node.instructions])))
for (nodeId,node) in nodeIdToGraphObjectMapping.items():
    for child in node.children:
        gra.edge(str(nodeId),str(child.nodeId))
#
gra

In [ ]:
from collections import deque


def DFS(currentNode:Node,executionStack:deque,mapVersionedVariables,mapLatestVersionForVariable):
    for instr_ in currentNode.instructions:
        match str(instr_.opname):
            case "LOAD_CONST":
                executionStack.append(instr.argval)
            case "STORE_FAST":
                var = str(instr.argval)
                if var in mapVersionedVariables:
                executionStack.pop()

In [ ]:
import z3
s = z3.Solver()

x1 = z3.Int('x@0')

# a = b + c

def binaryFunc():
    x1_ = z3.Int('x@1')


def func(x1)

In [ ]:
 x